In [1]:
import sys

import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels
from src.models import convnet_builder as bare_model_builder
from src.network_utils import count_params

PWD = '../../..'
sys.path.append(PWD)

tf.get_logger().setLevel('ERROR')

In [2]:
PROJECT_NAME = 'convnet'

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 500
BATCH_SIZE = 8096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all[(2, 11)].shape

((22134, 24), (22134,))

In [4]:
X_train, X_val, y_train, y_val = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train = np.concatenate(list(y_train.values()))
X_val = np.concatenate(list(X_val.values()))
y_val = np.concatenate(list(y_val.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train = X_train[train_idx], y_train[train_idx]
X_val, y_val = X_val[val_idx], y_val[val_idx]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((84824, 24), (84824,), (21208, 24), (21208,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=4, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 1)                 193       
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=30, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=5)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 24s]
val_loss: 9.277607440948486

Best val_loss So Far: 8.7764892578125
Total elapsed time: 00h 39m 51s


# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 14289
========== Model 1 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 107873
========== Model 2 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 55873
========== Model 3 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 3

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 14289
Got score: 8.6617 (8.6776, 8.6459)
Got score: 8.5027 (8.4411, 8.5643)
Got score: 8.4839 (8.4564, 8.5114)
Got score: 8.3823 (8.4287, 8.3358)
Got score: 8.6749 (8.7000, 8.6497)


{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 107873
Got score: 8.5064 (8.5092, 8.5035)
Got score: 15.9360 (23.7510, 8.1210)
Got score: 8.2954 (8.2213, 8.3694)
Got score: 8.4573 (8.4552, 8.4595)
Got score: 8.3095 (8.3789, 8.2401)


{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 55873
Got score: 8.7835 (8.9569, 8.6101)
Got score: 8.4765 (8.4830, 8.4699)
Got score: 8.4604 (8.4519, 8.4689)
Got score: 8.5898 (8.3573, 8.8223)
Got score: 8.5963 (8.6150, 8.5776)


{'n_conv_blocks': 1, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.2}
Number of parameters: 32321
Got score: 8.8036 (8.8154, 8.7918)
Got score: 8.8359 (8.6152, 9.0566)
Got score: 8.8285 (8.9417, 8.7154)
Got score: 8.5545 (8.9256, 8.1835)
Got score: 8.5051 (8.5166, 8.4937)


{'n_conv_blocks': 2, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.0}
Number of parameters: 721
Got score: 8.9294 (8.9252, 8.9337)
Got score: 8.5991 (8.6304, 8.5679)
Got score: 8.8073 (8.7593, 8.8553)
Got score: 8.5958 (8.6298, 8.5619)
Got score: 9.0115 (9.0254, 8.9977)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.54,0.11,14289
1,9.90,3.02,107873
2,8.58,0.12,55873
3,8.71,0.14,32321
4,8.79,0.17,721
